In [4]:
import sys; sys.path.append('../..')
from abslithist import *

In [10]:
C = lltk.load('CanonFiction')
# C.texts()

In [11]:
NLP_POS=None
def get_nlp_pos(force=True):
    #global NLP_POS
    if not force and NLP_POS is not None: return NLP_POS
    import stanza
    nlp = stanza.Pipeline('en', processors='tokenize,pos,lemma',verbose=False)
    if not force: NLP_POS = nlp
    return nlp

def get_text_pos(path_txt, lim_paras=100, progress=True, nlp_max_chars=3000):
    if not os.path.exists(path_txt): return pd.DataFrame()    

    import stanza
    nlp = stanza.Pipeline('en', processors='tokenize,pos,lemma',verbose=False)

    with open(path_txt) as f: text = f.read()
    paras = [para.strip() for para in text.split('\n\n')]
    paras = [' '.join(para.strip().split()) for para in paras if para]
    paras = paras[:lim_paras]    
    
    o=[]
    if progress: paras=tqdm(paras,position=0)
    for para_i,para in enumerate(paras):
        # if len(para)<500: continue
        if nlp_max_chars and len(para)>nlp_max_chars:
            para_sents = nltk.sent_tokenize(para)
            para_parts = []
            para_part_len=0
            para_part = []
            for para_sent in para_sents:
                para_part.append(para_sent)
                para_part_len+=len(para_sent)
                if para_part_len>=nlp_max_chars:
                    para_parts.append(' '.join(para_part))
                    para_part = []
                    para_part_len = 0
            if para_part: para_parts.append(' '.join(para_part))
        else:
            para_parts=[para]


        sent_i=0
        for parapart_i,para_part in enumerate(para_parts):
            # return para_part
            doc = nlp(para_part)
            for sent in doc.sentences:
                sent_i+=1
                toks = sent.to_dict()
                for tokd in toks:
                    tokd['sent_i']=sent_i
                    tokd['para_i']=para_i+1
                    # tokd['parapart_i']=parapart_i+1
                    if 'feats' in tokd:
                        for feat in tokd['feats'].split('|'):
                            if '=' in feat:
                                fk,fv = feat.split('=')
                                tokd[fk]=str(fv)
                        del tokd['feats']
                    o.append(tokd)
    return pd.DataFrame(o).rename({'id':'word_i'},axis=1).set_index(['para_i','sent_i','word_i']).fillna('')

In [12]:
# get_text_pos(C.t.path_txt)

In [13]:
def do_save_text_pos(obj,force=False,**kwargs):
    path_txt,path_pos = obj
    if not force and os.path.exists(path_pos): return
    odf = get_text_pos(path_txt, **kwargs)
    odir = os.path.dirname(path_pos)
    try:
        if not os.path.exists(odir):
            os.makedirs(odir)
    except FileExistsError:
        pass
    odf.to_pickle(path_pos)

def save_text_pos(C,lim=None,num_proc=1,force=False,lim_paras=100,progress_inner=True, nlp_max_chars=3000):
    kwargs=dict(progress=progress_inner, lim_paras=lim_paras, nlp_max_chars=nlp_max_chars, force=force)
    objs = [(t.path_txt, t.path_pos) for t in C.texts()][:lim]
    pmap(do_save_text_pos, objs, num_proc=num_proc, kwargs=kwargs)

In [15]:
C._texts = [t for t in C.texts() if not t.meta.get('corpus_source')]
len(C.texts())

112

In [17]:
save_text_pos(C, lim_paras=None, force=False, num_proc=4, lim=None)

112